In [9]:
# 데이터셋을 DataLoader로 변환하고, 모델을 정의하며, 학습 및 평가 루프를 직접 구현하는 과정

import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda
import timm

# 파이토치 기본 공식 베이직 코드를 참조함
# ds = datasets.FashionMNIST(
#     root="data",
#     train=True,
#     download=True,
#     transform=ToTensor(),
#     target_transform=Lambda(lambda y: torch.zeros(10, dtype=torch.float).scatter_(0, torch.tensor(y), value=1))
# )

# 원-핫 인코딩을 하는 과정
# '셔츠'를 나타내는 레이블 y = 6이 이 함수에 전달될 때 가정,
# torch.zeros(10, ...)이 [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.] 텐서를 생성.
# .scatter_ 메소드가 이 텐서를 받습니다. -> _의미: 제자리에서 작업이 이루어진다는 뜻
# torch.tensor(y)가 torch.tensor(6)이 되므로, 인덱스 6을 사용.
# 인덱스 6의 위치에 value=1을 넣습니다.
# 그 결과, [0., 0., 0., 0., 0., 0., 1., 0., 0., 0.]와 같은 텐서가 완성

In [10]:
# 데이터셋 준비
training_data = datasets.FashionMNIST(
  root="data",
  train=True,
  download=True,
  transform=ToTensor(),
  target_transform=Lambda(lambda y: torch.zeros(10, dtype=torch.float).scatter_(0, torch.tensor(y), value=1))
)

test_data = datasets.FashionMNIST(
  root="data",
  train=False,
  download=True,
  transform=ToTensor(),
  target_transform=Lambda(lambda y: torch.zeros(10, dtype=torch.float).scatter_(0, torch.tensor(y), value=1))
)



In [11]:
# DataLoader 생성
batch_size = 64
train_dataloader = DataLoader(training_data, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(test_data, batch_size=batch_size, shuffle=False)

In [13]:
# Vision Transformer (VIT)모델 정의
# ViT는 이미지를 패치로 분할한 후, 각 패치를 시퀀스로 다룹니다.
# cnn은 필터를 이동시켜 이미지의 지역적 특징을 추출하는 반면에 ViT는 이미지들을 작은 크기의 여러 패치로 나눈 후, 각 패치를 마치 문장의 단어처럼 다룹니다.
# 이 패치들의 순서 시퀀스를 트랜스포머 모델에 입력하여 전체 이미지의 특징을 학습합니다.

class SimpleVit(nn.Module):
  def __init__(self, in_channels=1, patch_size=4, embed_dim=128, num_layers=2, num_heads=4, num_classes=10):
    super().__init__()

    image_size = 28

    # 1. 패치 임베딩: 이미지를 패치로 나누고 각 패치를 벡터로 변환
    # 28x28 이미지를 4x4 크기의 겹치지 않는(stride=4) 조각들로 나눕니다. 이렇게 만들어진 각 패치는 embed_dim (128) 차원의 벡터로 변환.
    self.patch_embed = nn.Conv2d(in_channels, embed_dim, kernel_size=patch_size, stride=patch_size)

    # 2. 위치 인코딩: 패치들의 순서 정보를 추가
    # num_patches + 1은 패치 개수(28/4 * 28/4 = 49)에 특별한 [CLS] 토큰 한 개를 더한 것입니다.
    num_patches = (image_size // patch_size) * (image_size // patch_size)
    self.positions = nn.Parameter(torch.randn(num_patches + 1, embed_dim))

    # 3.[CLS] 토큰: 시퀀스 전체를 대표하는 특별한 토큰
    # 트랜스포머 인코더는 이 토큰을 통해 전체 패치들의 정보를 한곳에 모읍니다. 학습이 끝나면 이 [CLS] 토큰의 출력이 전체 이미지를 대표하는 벡터가 됩니다. 우리는 이 벡터를 분류에 사용
    self.cls_taken = nn.Parameter(torch.randn(1, 1, embed_dim))

    # 4. 트랜스포머 인코더: 패치 간의 관계를 학습
    # nhead=num_heads 어텐션 헤드의 수 -> 어텐션 메커니즘이 한 번에 여러 관점에서 패치 간의 관계를 탐색
    encoder_layer = nn.TransformerEncoderLayer(d_model=embed_dim, nhead=num_heads, batch_first=True)

    # 5. 분류 헤드: 최종 예측을 수행합니다.
    self.classifier = nn.Linear(embed_dim, num_classes)

  def forward(self, x):
    # 이미지를 패치로 변환 (N, C, H, W) -> (N, P, E)
    # flatten(2): 패치들을 1차원 시퀀스로 펼칩니다.
    # transpose(1, 2):차원의 순서를 바꿉니다. 트랜스포머 입력에 맞는 (배치_크기, 패치_개수, 임베딩_차원) 형태로 만듭니다.
    x = self.patch_embed(x)
    x = x.flatten(2).transpose(1, 2)

    # [CLS] 토큰을 패치 시퀀스에 추가
    cls_tokens = self.cls_token.expand(x.shape[0], -1, -1). # 0은 미니배치 데이터, -1은 해당 차원의 크기를 그대로
    x = torch.cat((cls_tokens, x), dim=1)

    # 위치 임베딩 추가
    x += self.positions

    # 트랜스포머 인코더에 통과
    x = self.Transformer_encoder(x)

    # [CLS] 토큰의 출력을 사용하여 분류
    cls_taken_output = x[:, 0]
    logits = self.classifier(cls_taken_output)
    return logits

# 모델 인스턴스 생성
model = SimpleVit()

# 이미지 -> (패치로 분할) -> [CLS] 토큰과 위치 정보가 추가된 패치 시퀀스 -> 트랜스포머 인코더에 통과 -> [CLS] 토큰의 출력을 사용 -> 분류기에서 최종 예측.